# RoboJam Training with Metatone Data

In [ ]:
import keras
from keras import backend as K
from keras.layers import Dense, Input
import numpy as np
import tensorflow as tf
import math
import h5py
import random
import time
import pandas as pd
from context import * # imports MDN
import matplotlib.pyplot as plt
%matplotlib inline

input_colour = 'darkblue'
gen_colour = 'firebrick'
plt.style.use('seaborn-talk')

## Helper functions for touchscreen performances

We need a few helper functions for managing performances:
    
- Convert performances to and from pandas dataframes.
- Generate random touches.
- Sample whole performances from scratch and from a priming performance.
- Plot performances including dividing into swipes.

# Load up the Dataset:

The dataset consists of around 1000 5-second performances from the MicroJam app.

This is in a sequence of points consisting of an x-location, a y-location, and a time-delta from the previous point.

When the user swipes, the time-delta is very small, if they tap it's quite large.

Let's have a look at some of the data:

In [ ]:
# Load Data
microjam_data_file_name = "../datasets/TinyPerformanceCorpus.h5"
microjam_data_file_name = "../datasets/MetatoneTinyPerformanceRecords.h5"


with h5py.File(microjam_data_file_name, 'r') as data_file:
    microjam_corpus = data_file['total_performances'][:]

print("Corpus data points between 100 and 120:")
print(perf_array_to_df(microjam_corpus[100:120]))

print("Some statistics about the dataset:")
pd.DataFrame(microjam_corpus).describe()

- This time, the X and Y locations are *not* differences, but the exact value, but the time is a delta value.
- The data doesn't have a "pen up" value, but we can just call taps with dt>0.1 as taps, dt<0.1 as moving touches.

In [ ]:
# Plot a bit of the data to have a look:
plot_2D(perf_array_to_df(microjam_corpus[100:200]))

# Network

In [ ]:
# Training Hyperparameters:
SEQ_LEN = 30
BATCH_SIZE = 256
HIDDEN_UNITS = 256
EPOCHS = 100
VAL_SPLIT=0.15

# Set random seed for reproducibility
SEED = 2345  
random.seed(SEED)
np.random.seed(SEED)

def slice_sequence_examples(sequence, num_steps):
    xs = []
    for i in range(len(sequence) - num_steps - 1):
        example = sequence[i: i + num_steps]
        xs.append(example)
    return xs

def seq_to_singleton_format(examples):
    xs = []
    ys = []
    for ex in examples:
        xs.append(ex[:-1])
        ys.append(ex[-1])
    return (xs,ys)

sequences = slice_sequence_examples(microjam_corpus, SEQ_LEN+1)
print("Total training examples:", len(sequences))
X, y = seq_to_singleton_format(sequences)
X = np.array(X)
y = np.array(y)
print("X:", X.shape, "y:", y.shape)

Now let's set up the model:

In [ ]:
OUTPUT_DIMENSION = 3
NUMBER_MIXTURES = 5

model = keras.Sequential()
model.add(keras.layers.LSTM(HIDDEN_UNITS, batch_input_shape=(None,SEQ_LEN,OUTPUT_DIMENSION), return_sequences=True))
model.add(keras.layers.LSTM(HIDDEN_UNITS))
model.add(mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES))
model.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer=keras.optimizers.Adam())
model.summary()

# Training

In [ ]:
# Train the model
history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT)

# Save the Model
model.save('robojam-mdn-rnn-metatone.h5')  # creates a HDF5 file of the model

# Plot the loss
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# Plot the loss
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.ylim(-15,0)
plt.show()

# maybe just try training to 30 epochs

# Testing the model

In [ ]:
# Decoding Model
decoder = keras.Sequential()
decoder.add(keras.layers.LSTM(HIDDEN_UNITS, batch_input_shape=(1,1,OUTPUT_DIMENSION), return_sequences=True, stateful=True))
decoder.add(keras.layers.LSTM(HIDDEN_UNITS, stateful=True))
decoder.add(mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES))
decoder.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer=keras.optimizers.Adam())
decoder.summary()

# decoder.set_weights(model.get_weights())
decoder.load_weights("robojam-mdn-rnn.h5")

Plotting some conditioned performances.

This model seems to work best with a very low temperature (0.1). Might be able to do better with a large dataset, or larger model! (?)

In [ ]:
length = 100
t = random.randint(0,len(microjam_corpus)-length)
ex =  microjam_corpus[t:t+length]  #sequences[600]

decoder.reset_states()
p = robojam.condition_and_generate(decoder, ex, NUMBER_MIXTURES, temp=0.2)
plot_double_2d(robojam.perf_array_to_df(ex), robojam.perf_array_to_df(p))

We can also generate unconditioned performances from a random starting point.

In [ ]:
decoder.reset_states()
t = robojam.random_touch()
p = robojam.generate_random_tiny_performance(decoder, NUMBER_MIXTURES, t, temp=0.1)
plot_2D(robojam.perf_array_to_df(p))